In [1]:
!pip install evaluate
!pip install transformers
!pip install bert_score
!pip install sentence_transformers
!pip install tensorflow_hub
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 19.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/sim

In [2]:
from evaluate import load
import pandas as pd
from ast import literal_eval
from sentence_transformers import CrossEncoder
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import numpy as np
from scipy.special import softmax


In [3]:
#setup globals for bem tokenization, code from https://colab.research.google.com/github/google-research-datasets/answer-equivalence-dataset/blob/main/Answer_Equivalence_BEM_example.ipynb#scrollTo=NND0honxyDaJ
VOCAB_PATH = 'gs://cloud-tpu-checkpoints/bert/keras_bert/uncased_L-12_H-768_A-12/vocab.txt'

vocab_table = tf.lookup.StaticVocabularyTable(
        tf.lookup.TextFileInitializer(
            filename=VOCAB_PATH,
            key_dtype=tf.string,
            key_index=tf.lookup.TextFileIndex.WHOLE_LINE,
            value_dtype=tf.int64,
            value_index=tf.lookup.TextFileIndex.LINE_NUMBER
        ), 
        num_oov_buckets=1)
cls_id, sep_id = vocab_table.lookup(tf.convert_to_tensor(['[CLS]', '[SEP]']))
tokenizer = text.BertTokenizer(vocab_lookup_table=vocab_table, 
                               token_out_type=tf.int64, 
                               preserve_unused_token=True, 
                               lower_case=True)

In [4]:
bertscore = load("bertscore")
SAS = CrossEncoder('cross-encoder/stsb-roberta-large')
bem = hub.load('https://tfhub.dev/google/answer_equivalence/bem/1')


In [5]:
def compute_bertscore_f1(references, prediction):
    predictions = [prediction] * len(references)
    results = bertscore.compute(predictions=predictions, references=references, model_type="roberta-large") 
    return max(results['f1'])

In [6]:
def compute_SAS_scores(references, prediction):
    predictions = [prediction] * len(references)
    inputs = list(zip(references, predictions))
    scores = SAS.predict(inputs)
    return max(scores)

In [7]:
#helper functions for the bem model code from https://colab.research.google.com/github/google-research-datasets/answer-equivalence-dataset/blob/main/Answer_Equivalence_BEM_example.ipynb#scrollTo=S_4PBlU5yj3f
def bertify_example(example):
    question = tokenizer.tokenize(example['question']).merge_dims(1, 2)
    reference = tokenizer.tokenize(example['reference']).merge_dims(1, 2)
    candidate = tokenizer.tokenize(example['candidate']).merge_dims(1, 2)

    input_ids, segment_ids = text.combine_segments(
        (candidate, reference, question), cls_id, sep_id)

    return {'input_ids': input_ids.numpy(), 'segment_ids': segment_ids.numpy()}


def pad(a, length=512):
    return np.append(a, np.zeros(length - a.shape[-1], np.int32))


def bertify_examples(examples):
    input_ids = []
    segment_ids = []
    for example in examples:
        example_inputs = bertify_example(example)
        input_ids.append(pad(example_inputs['input_ids']))
        segment_ids.append(pad(example_inputs['segment_ids']))

    return {'input_ids': np.stack(input_ids), 'segment_ids': np.stack(segment_ids)}

def predict_example(examples):
    inputs = bertify_examples(examples)
    raw_outputs = bem(inputs)
    return float(softmax(np.squeeze(raw_outputs))[1])    

def compute_BEM_scores(question, references, prediction):
    equivalences = []
    for reference in references:
        data = [{'question': question, 'reference': reference, 'candidate': prediction}]
        equivalences.append(predict_example(data))
    return max(equivalences)

In [8]:
examples = [{
    'question': 'why is the sky blue',
    'reference': 'light scattering',
    'candidate': 'scattering of light'
    }]

inputs = bertify_examples(examples)

# The outputs are raw logits.
raw_outputs = bem(inputs)

# They can be transformed into a classification 'probability' like so:
bem_score = float(softmax(np.squeeze(raw_outputs))[1])

print(f'BEM score: {bem_score}')

BEM score: 0.9891805052757263


In [9]:

df = pd.read_csv("/content/answer_pairs2.csv")
df['Gold Answers'] = df['Gold Answers'].apply(literal_eval)


In [10]:
df['BERTScore Similarities GPT'] = df.apply(lambda row: compute_bertscore_f1(row['Gold Answers'], row['GPT Answer']), axis=1)


In [11]:
df['SAS Similarities GPT'] = df.apply(lambda row: compute_SAS_scores(row['Gold Answers'], row['GPT Answer']), axis=1)

In [12]:
df['BEM Similarities GPT'] = df.apply(lambda row: compute_BEM_scores(row['Questions'], row['Gold Answers'], row['GPT Answer']), axis=1)

In [13]:
df.head()

,Unnamed: 0,Contexts,Questions,Gold Answers,GPT Answer,LERC Score GPT,Annotator Label - GPT Answer,Category,BERTScore Similarities GPT,SAS Similarities GPT,BEM Similarities GPT
0,0,"['Do - Right , segment of The Rocky and Bullwi...",who was the bad guy in dudley do right,[Snidely Whiplash],Snidely Whiplash,5.081576,3,Exact match,1.000000,0.966667,0.989872
1,1,['Martin Deborah Knox Fiona MacDonald Janice R...,who won the women's gold medal curling,[Sweden (],It depends on the specific year and competitio...,1.372812,1,DATC,0.806277,0.039457,0.057492
2,2,[' The Dome of the Rock ( Arabic : قبة الصخ...,who built the dome of the rock mosque,[initially completed in 691 CE at the order of...,The dome of the rock mosque was built by Calip...,3.717231,3,Semantically the same,0.911975,0.656942,0.823226
3,3,['Chevrolet Zafira compact MPV . Chevrolet m...,what does hhr stand for on a chevy,[Heritage High Roof],HHR stands for Heritage High Roof.,4.726677,3,Semantically the same,0.812078,0.838378,0.989761
4,4,"[""was mentored by one of the show 's four judg...",who won x factor when olly murs came second,[Joe McElderry],Joe McElderry,5.079546,3,Exact match,1.000000,0.963869,0.991353


In [14]:
df.to_csv("answer_pairs2.csv", index=False)